In [7]:
import pymongo

MONGODB_HOST = 'localhost'
MONGODB_PORT = '27017'
MONGODB_TIMEOUT = 1000

URI_CONNECTION = "mongodb://" + MONGODB_HOST + ":" + MONGODB_PORT +  "/"

try:
     client = pymongo.MongoClient(URI_CONNECTION)
     client.server_info()
     print ('OK -- Connected to MongoDB at server %s' % (MONGODB_HOST))
     client.close()
except pymongo.errors.ServerSelectionTimeoutError as error:
        print ('Error with MongoDB connection: %s' % error)
except pymongo.errors.ConnectionFailure as error:
        print ('Could not connect to MongoDB: %s' % error)

    


OK -- Connected to MongoDB at server localhost


In [1]:
pip install Scrapy

Note: you may need to restart the kernel to use updated packages.


In [4]:
from scrapy.item import Field
from scrapy.item import Item
from scrapy.spiders import CrawlSpider, Rule
from scrapy.selector import Selector
from scrapy.loader.processors import MapCompose
from scrapy.linkextractors import LinkExtractor
from scrapy.loader import ItemLoader
#from bs4 import BeautifulSoup
from scrapy.crawler import CrawlerProcess

In [13]:
class Articulo(Item):
    titulo = Field()
    precio = Field()
    descripcion = Field()

class daneCrawler(CrawlSpider):
    name = 'dane'

    custom_settings = {
      'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36',
      'CLOSESPIDER_PAGECOUNT': 20 # Numero maximo de paginas en las cuales voy a descargar items. Scrapy se cierra cuando alcanza este numero
    }

    # Utilizamos 2 dominios permitidos, ya que los articulos utilizan un dominio diferente
    allowed_domains = ['dane.gov.co', 'www.dane.gov.co']

    start_urls = ['https://www.dane.gov.co/index.php/estadisticas-por-tema/industria/encuesta-anual-manufacturera-enam/eam-historicos']

    download_delay = 1

    # Tupla de reglas
    rules = (
        Rule( # REGLA #1 => HORIZONTALIDAD POR PAGINACION
            LinkExtractor(
                allow=r'/_Desde_\d+' # Patron en donde se utiliza "\d+", expresion que puede tomar el valor de cualquier combinacion de numeros
            ), follow=True),
        Rule( # REGLA #2 => VERTICALIDAD AL DETALLE DE LOS PRODUCTOS
            LinkExtractor(
                allow=r'/MCO' 
            ), follow=True, callback='parse_items'), # Al entrar al detalle de los productos, se llama al callback con la respuesta al requerimiento
    )

    def parse_items(self, response):

        item = ItemLoader(Articulo(), response)
            
        item.add_xpath('Período de referencia', '/html/body/div[1]/div[3]/div/div[1]/div/article/section/table[1]/tbody/tr[5]/td[1]/span/strong', MapCompose(lambda i: i.replace('\n', ' ').replace('\r', ' ').strip()))
        item.add_xpath('Anexos', '//*[@id="t3-content"]/div/article/section/table[1]/tbody/tr[2]/td[4]/ul'.replace('\r', ' ').strip())
        
       
        yield item.load_item()


In [14]:
# CORRIENDO SCRAPY SIN LA TERMINAL
process = CrawlerProcess({
     'FEED_FORMAT': 'csv',
     'FEED_URI': 'datos_de_salida6.csv'
 })
process.crawl(daneCrawler)
process.start()

2022-09-22 22:26:26 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2022-09-22 22:26:26 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Windows-10-10.0.19044-SP0
2022-09-22 22:26:26 [scrapy.crawler] INFO: Overridden settings:
{'CLOSESPIDER_PAGECOUNT': 20,
 'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, '
               'like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 '
               'Safari/537.36'}
2022-09-22 22:26:26 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-09-22 22:26:26 [scrapy.extensions.telnet] INFO: Telnet Password: d9860dc2aef7be3a
2022-09-22 22:26:26 [py.warnings] WARNING: c:\Users\Edison\AppData\Local\Programs\Python\Python310\lib\site